In [39]:
import numpy as np
import pandas as pd
from pandas_datareader import data as web
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
 
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
cf.go_offline()

import warnings
warnings.simplefilter("ignore")
import yfinance as yf

from plotly.subplots import make_subplots

from ta.trend import MACD
from ta.momentum import StochasticOscillator
import ta as ta

In [40]:
# Relative Strength Index
#Over bought and oversold
#Potential changes and different changes

amd_df=yf.download(tickers="AMD",period="3d",interval="15m")
amd_df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-03-09 09:30:00-05:00,108.410004,109.349899,107.349998,107.669998,107.669998,10107396
2022-03-09 09:45:00-05:00,108.000000,108.129997,106.860001,108.019897,108.019897,6449700
2022-03-09 10:00:00-05:00,108.019897,109.849998,107.800003,109.570000,109.570000,6665283
2022-03-09 10:15:00-05:00,109.570000,109.970001,108.480003,109.320000,109.320000,5561847
2022-03-09 10:30:00-05:00,109.330002,110.250000,108.940002,109.459999,109.459999,4823447
...,...,...,...,...,...,...
2022-03-11 15:00:00-05:00,105.792397,105.980003,105.370003,105.505898,105.505898,2621075
2022-03-11 15:15:00-05:00,105.500000,105.799004,105.129997,105.175003,105.175003,2225814
2022-03-11 15:30:00-05:00,105.160004,105.279999,104.820000,104.870003,104.870003,2494065


In [48]:
delta =amd_df['Close'].diff(1)[1:]
figsize=(12.2,4.5)
positive=delta.copy()
negative= delta.copy()
positive[positive < 0] = 0
negative[negative > 0] = 0
time_period = 20
AVG_Gain = positive.rolling(window=time_period).mean()
Avg_loss = abs(negative.rolling(window=time_period).mean())
Relative_strength=AVG_Gain / Avg_loss
amd_df['RSI']=100.0 - (100.0/(1.0 + Relative_strength))
amd_df

,Open,High,Low,Close,Adj Close,Volume,RSI
Datetime,,,,,,,
2022-03-09 09:30:00-05:00,108.410004,109.349899,107.349998,107.669998,107.669998,10107396,NaN
2022-03-09 09:45:00-05:00,108.000000,108.129997,106.860001,108.019897,108.019897,6449700,NaN
2022-03-09 10:00:00-05:00,108.019897,109.849998,107.800003,109.570000,109.570000,6665283,NaN
2022-03-09 10:15:00-05:00,109.570000,109.970001,108.480003,109.320000,109.320000,5561847,NaN
2022-03-09 10:30:00-05:00,109.330002,110.250000,108.940002,109.459999,109.459999,4823447,NaN
...,...,...,...,...,...,...,...
2022-03-11 15:00:00-05:00,105.792397,105.980003,105.370003,105.505898,105.505898,2621075,45.423199
2022-03-11 15:15:00-05:00,105.500000,105.799004,105.129997,105.175003,105.175003,2225814,44.952734
2022-03-11 15:30:00-05:00,105.160004,105.279999,104.820000,104.870003,104.870003,2494065,40.443789


In [45]:
#Plot RSI
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                    vertical_spacing=0.01,
                   row_heights=[0.7, 0.3])

candle = go.Candlestick(x=amd_df.index, open=amd_df['Open'],
                       high=amd_df['High'], low=amd_df['Low'],
                       close=amd_df['Close'], name='Candlestick')

rsi = go.Scatter(x=amd_df.index, y=amd_df['RSI'], 
                 line=dict(color='blue', width=2))

fig.add_trace(candle, row=1, col=1)  
fig.add_trace(rsi, row=2, col=1) 

# Draw 20 and 80 lines
fig.add_hline(y=30, line_width=1, line_dash="dash", line_color="red", row=2, col=1)
fig.add_hline(y=70, line_width=1, line_dash="dash", line_color="green", row=2, col=1)

# Add title
fig.update_layout(title="AMD")

# Get rid of empty dates and market closed
fig.update_layout(height=900, width=1200, 
                  showlegend=False, 
                  xaxis_rangeslider_visible=False,
                  xaxis_rangebreaks=[
            dict(bounds=["sat", "mon"]),
            dict(bounds=[16, 9.5], pattern="hour")
        ])
                                            
fig.show()

In [56]:
#Bollinger Bands, we are going to plot  two line using as moving average and standard deviasion going to figure ourt how a part the lines  are.
# High Band(Resistance)= Simple Moving Average + 2*Standard Deviation
# Low Band(Support)= Simple Moving Average +2*Standard Deviation
amd_df=yf.download(tickers="AMD",period="6mo",interval="1d")
amd_df['Mean']=amd_df['Close'].rolling(window=20).mean()
amd_df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Mean
Date,,,,,,,
2021-09-13,105.260002,105.279999,102.629997,104.800003,104.800003,37767800,NaN
2021-09-14,105.519997,106.820000,104.419998,105.730003,105.730003,40629500,NaN
2021-09-15,106.000000,106.320000,104.029999,105.599998,105.599998,37998900,NaN
2021-09-16,104.919998,106.750000,103.760002,106.220001,106.220001,38150300,NaN
2021-09-17,105.559998,105.980003,103.709999,103.879997,103.879997,44528300,NaN
...,...,...,...,...,...,...,...
2022-03-07,108.529999,109.529999,102.820000,102.949997,102.949997,92600000,117.260000
2022-03-08,102.809998,109.900002,100.080002,105.529999,105.529999,135348300,116.353000
2022-03-09,108.410004,111.709999,106.849998,111.050003,111.050003,102310300,115.494001


In [57]:
amd_df['SD']=amd_df['Close'].rolling(window=20).std()
amd_df

,Open,High,Low,Close,Adj Close,Volume,Mean,SD
Date,,,,,,,,
2021-09-13,105.260002,105.279999,102.629997,104.800003,104.800003,37767800,NaN,NaN
2021-09-14,105.519997,106.820000,104.419998,105.730003,105.730003,40629500,NaN,NaN
2021-09-15,106.000000,106.320000,104.029999,105.599998,105.599998,37998900,NaN,NaN
2021-09-16,104.919998,106.750000,103.760002,106.220001,106.220001,38150300,NaN,NaN
2021-09-17,105.559998,105.980003,103.709999,103.879997,103.879997,44528300,NaN,NaN
...,...,...,...,...,...,...,...,...
2022-03-07,108.529999,109.529999,102.820000,102.949997,102.949997,92600000,117.260000,7.220304
2022-03-08,102.809998,109.900002,100.080002,105.529999,105.529999,135348300,116.353000,7.506400
2022-03-09,108.410004,111.709999,106.849998,111.050003,111.050003,102310300,115.494001,7.044505


In [58]:
amd_df['BB_Hi']=amd_df['Mean']+ (2 * amd_df['SD'])
amd_df['BB_Low']=amd_df['Mean']- (2 * amd_df['SD'])
amd_df

,Open,High,Low,Close,Adj Close,Volume,Mean,SD,BB_Hi,BB_Low
Date,,,,,,,,,,
2021-09-13,105.260002,105.279999,102.629997,104.800003,104.800003,37767800,NaN,NaN,NaN,NaN
2021-09-14,105.519997,106.820000,104.419998,105.730003,105.730003,40629500,NaN,NaN,NaN,NaN
2021-09-15,106.000000,106.320000,104.029999,105.599998,105.599998,37998900,NaN,NaN,NaN,NaN
2021-09-16,104.919998,106.750000,103.760002,106.220001,106.220001,38150300,NaN,NaN,NaN,NaN
2021-09-17,105.559998,105.980003,103.709999,103.879997,103.879997,44528300,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-03-07,108.529999,109.529999,102.820000,102.949997,102.949997,92600000,117.260000,7.220304,131.700607,102.819393
2022-03-08,102.809998,109.900002,100.080002,105.529999,105.529999,135348300,116.353000,7.506400,131.365800,101.340200
2022-03-09,108.410004,111.709999,106.849998,111.050003,111.050003,102310300,115.494001,7.044505,129.583010,101.404991


In [59]:
#Plot RSI
fig = go.Figure()

candle = go.Candlestick(x=amd_df.index, open=amd_df['Open'],
                       high=amd_df['High'], low=amd_df['Low'],
                       close=amd_df['Close'], name='Candlestick')

bb_hi = go.Scatter(x=amd_df.index, y=amd_df['BB_Hi'], 
                 line=dict(color='blue', width=1),name='BB Hi')

bb_low = go.Scatter(x=amd_df.index, y=amd_df['BB_Low'], 
                 line=dict(color='blue', width=1),name='BB Low')


fig.add_trace(candle)
fig.add_trace(bb_hi) 
fig.add_trace(bb_low) 

# Draw 20 and 80 lines
fig.add_hline(y=30, line_width=1, line_dash="dash", line_color="red", row=2, col=1)
fig.add_hline(y=70, line_width=1, line_dash="dash", line_color="green", row=2, col=1)

# Add title
fig.update_layout(title="AMD")

# Get rid of empty dates and market closed
fig.update_xaxes(rangebreaks=[dict(bounds=["sat","mon"])])
fig.show()